### Eriks dubbelkoll

Jag tänker att jag skriver simuleringar där jag dubbelkollar det vi skriver i rapporten.

### Imports

In [1]:
import ROOT
import time

Welcome to JupyROOT 6.22/02


### Class definitions

In [2]:
class noise_generator:
    def __init__(self, threshold, channel_count):
        noise = ROOT.TF1("noise_gauss", "TMath::Gaus(x, 0, 5)", 0, 1000)
        self._mean = channel_count * noise.Integral(threshold, 1000)/noise.Integral(0, 1000)/2
        self._prng = ROOT.TRandom3(time.time_ns())

    def generate(self):
        total = self._prng.Poisson(self._mean)
        narrow = 0
        for i in range(total):
            if self._prng.Uniform(25) < 3.125:
                narrow += 1
        estimate = 3.125 * (total - narrow)/(25 - 3.125)
        return narrow, estimate

    
class particles_generator:
    def __init__(self, threshold, expected_voltage):
        self._threshold = threshold
        self._expected_voltage = expected_voltage
        signal_func = ROOT.TF1("sth", "TMath::Landau(x, %f, %f)" % (expected_voltage, expected_voltage/4), -100, 1000)
        self.fraction_above = signal_func.Integral(threshold, 1000)/signal_func.Integral(-100, 1000)
        self._prng = ROOT.TRandom3(time.time_ns())
    
    def generate(self, expected_count):
        signal_count = self._prng.Poisson(expected_count)
        above_threshold_count = 0
        for i in range(signal_count):
            if self._prng.Landau(self._expected_voltage, self._expected_voltage/4) > self._threshold:
                above_threshold_count += 1
        return signal_count, above_threshold_count
    
class simulation:
    def __init__(self, threshold, expected_voltage, expected_count, bunch_crossing_count):
        self._threshold = threshold
        self._expected_voltage = expected_voltage
        self._expected_count = expected_count
        self._bunch_crossing_count = bunch_crossing_count
        
        channel_count = int(3.6 * 1e6)
        self._particles = particles_generator(threshold, expected_voltage)
        self._noise = noise_generator(threshold, channel_count)
    
    def run(self):
        actual_particles_sum = 0
        estimated_particles_sum = 0

        for i in range(self._bunch_crossing_count):
            noise_exact, noise_est = self._noise.generate()
            particles_total, particles_above = self._particles.generate(self._expected_count)

            total_activations = particles_above + noise_exact
            particles_est = (total_activations - noise_est)/self._particles.fraction_above

            actual_particles_sum += particles_total
            estimated_particles_sum += particles_est

        return estimated_particles_sum/actual_particles_sum

### Simulations

In [3]:
my_sim = simulation(25, 100, 30, 1000000)
t = time.time()
print(my_sim.run())
t = time.time() - t
print(t)

1.0000208600855314
18.711135625839233


### Kommentarer på resultat
Som väntat går relativa felet mot noll då antalet bunch crossings ökar. Är det vi håller på med ens meningsfullt då?